In [ ]:
# This cell is auto-generated by Garden. Don't delete it. Do keep it as the first cell.
# It records the base image and requirements you passed to `garden-ai notebook start`.
# That way the next time you run this notebook Garden can start it with the same libraries.

"""
NOTEBOOK_METADATA = {
  "NOTEBOOK_BASE_IMAGE_NAME": "3.10-torch",
  "NOTEBOOK_REQUIREMENTS": null
}
"""

In [10]:
from garden_ai.model_connectors import HFConnector
from garden_ai import EntrypointMetadata, garden_entrypoint, entrypoint_test

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
my_hugging_face_repo = HFConnector("willengler-uc/torch-mnist")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

In [6]:
my_entrypoint_meta = EntrypointMetadata(
    title="MNIST Digit Classsifier",
    description="Example of a PyTorch model in Garden, using the MNIST digit classification problem",
    authors=["Will Engler"],
)

In [8]:
@garden_entrypoint(metadata=my_entrypoint_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.26311/b74a-5c58")
def classify_digits_mnist(image_batch):    
    download_path = my_hugging_face_repo.stage()
    from mnist_model import Net
    model = Net()
    model.load_state_dict(torch.load(download_path + "/mnist_model.pt"))
    model.eval()

    with torch.no_grad():
        output = model(image_batch)
        as_digits = output.data.max(1)[1]

    return as_digits

In [14]:
@entrypoint_test(classify_digits_mnist)
def test_digit_classifier():
    
    def get_a_test_image_batch():
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        data, target = next(iter(test_loader))
        return data, target
    
    test_image_batch, expected_result_batch = get_a_test_image_batch()
    classified_digits = classify_digits_mnist(test_image_batch)
    print (f"Classified {len(classified_digits)} digits")
    print("Accuracy:", (classified_digits == expected_result_batch).sum()/len(classified_digits))

In [15]:
test_digit_classifier()

100% 9912422/9912422 [00:00<00:00, 13542718.11it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100% 28881/28881 [00:00<00:00, 23869102.23it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100% 1648877/1648877 [00:00<00:00, 11719069.03it/s]


...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

mnist_model.py:   0%|          | 0.00/570 [00:00<?, ?B/s]

mnist_model.pt:   0%|          | 0.00/90.4k [00:00<?, ?B/s]

Classified 64 digits
Accuracy: tensor(0.9844)
